In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.utils import save_image
import os
from PIL import Image
import numpy as np

In [2]:
print(torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

True
Using device: cuda


In [3]:
class LOLDataset(Dataset):
    def __init__(self, low_dir, high_dir, transform=None):
        self.low_dir = low_dir
        self.high_dir = high_dir
        self.transform = transform

        self.low_images = [f for f in os.listdir(low_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))]
        self.high_images = [f for f in os.listdir(high_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))]

        assert len(self.low_images) == len(self.high_images), "Low and High images count mismatch."

    def __len__(self):
        return len(self.low_images)

    def __getitem__(self, idx):
        low_img_path = os.path.join(self.low_dir, self.low_images[idx])
        high_img_path = os.path.join(self.high_dir, self.high_images[idx])

        low_img = Image.open(low_img_path).convert("RGB")
        high_img = Image.open(high_img_path).convert("RGB")

        if self.transform:
            low_img = self.transform(low_img)
            high_img = self.transform(high_img)

        return low_img, high_img

In [4]:
def get_dataloaders(batch_size=8):
    transform = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
    ])
    
    dataset_path = "C://Users//kalya//Downloads//LOLdataset//"
    train_low_dir = os.path.join(dataset_path, 'our485//low//')
    train_high_dir = os.path.join(dataset_path, 'our485//high//')

    train_dataset = LOLDataset(low_dir=train_low_dir, high_dir=train_high_dir, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    return train_loader


In [5]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.layer1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.layer2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.layer3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(128 * 64 * 64, 512)
        self.fc2 = nn.Linear(512, 3 * 64 * 64)
        
    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.relu(self.layer3(x))
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        x = x.view(-1, 3, 64, 64)
        return x


In [6]:
model = SimpleCNN().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [7]:
def train(model, train_loader, num_epochs=20, save_path="C://Users//kalya//Downloads//Test//Pretrained Weight//model_weights.pth"):
    torch.cuda.empty_cache()
    model.train()
    
    for epoch in range(num_epochs):
        epoch_loss = 0.0
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

            if batch_idx % 10 == 0:
                print(f"Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx}/{len(train_loader)}], Loss: {loss.item():.4f}")

        torch.save(model.state_dict(), save_path)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(train_loader):.4f}")
        print(f"Model weights saved to {save_path}")

In [8]:
train_loader = get_dataloaders(batch_size=8)


In [9]:
torch.cuda.empty_cache()
train(model, train_loader)

Epoch [1/20], Step [0/61], Loss: 0.2832
Epoch [1/20], Step [10/61], Loss: 0.1403
Epoch [1/20], Step [20/61], Loss: 0.0762
Epoch [1/20], Step [30/61], Loss: 0.0447
Epoch [1/20], Step [40/61], Loss: 0.0799
Epoch [1/20], Step [50/61], Loss: 0.0593
Epoch [1/20], Step [60/61], Loss: 0.0458
Epoch [1/20], Loss: 0.1479
Model weights saved to C://Users//kalya//Downloads//Test//Pretrained Weight//model_weights.pth
Epoch [2/20], Step [0/61], Loss: 0.0316
Epoch [2/20], Step [10/61], Loss: 0.0468
Epoch [2/20], Step [20/61], Loss: 0.0386
Epoch [2/20], Step [30/61], Loss: 0.0354
Epoch [2/20], Step [40/61], Loss: 0.0430
Epoch [2/20], Step [50/61], Loss: 0.0469
Epoch [2/20], Step [60/61], Loss: 0.0457
Epoch [2/20], Loss: 0.0453
Model weights saved to C://Users//kalya//Downloads//Test//Pretrained Weight//model_weights.pth
Epoch [3/20], Step [0/61], Loss: 0.0375
Epoch [3/20], Step [10/61], Loss: 0.0487
Epoch [3/20], Step [20/61], Loss: 0.0476
Epoch [3/20], Step [30/61], Loss: 0.0462
Epoch [3/20], Step [4

In [10]:
test_dir = "C://Users//kalya//Downloads//Test//Test//Fusion//"
weights_path = "C://Users//kalya//Downloads//Test//Pretrained Weight//model_weights.pth"
output_dir = "C://Users//kalya//Downloads//Enhanced Low Light Images//"


In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleCNN().to(device)
model.load_state_dict(torch.load(weights_path, map_location=device))
model.eval()

C:\Users\kalya\AppData\Local\Temp\ipykernel_15140\1740479061.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(weights_path, map_location=

SimpleCNN(
  (layer1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu): ReLU()
  (layer2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (layer3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=524288, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=12288, bias=True)
)

In [12]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

In [13]:
def enhance_image(image_path):
    image = Image.open(image_path).convert('RGB')
    input_tensor = transform(image).unsqueeze(0).to(device) 
    with torch.no_grad():
        enhanced_tensor = model(input_tensor) 
    enhanced_image = enhanced_tensor.squeeze(0).cpu().numpy()
    enhanced_image = np.transpose(enhanced_image, (1, 2, 0)) 
    enhanced_image = (enhanced_image * 255).astype(np.uint8) 
    return Image.fromarray(enhanced_image)

In [14]:
for image_name in os.listdir(test_dir):
    if image_name.endswith(('.png', '.jpg', '.jpeg')): 
        image_path = os.path.join(test_dir, image_name)
        enhanced_image = enhance_image(image_path)
        
        # Save the enhanced image
        save_path = os.path.join(output_dir, image_name)
        enhanced_image.save(save_path)

print("Enhancement of low-light images complete!")

Enhancement of low-light images complete!
